# COGS 108 - EDA Checkpoint

# Names

- Clara Yi
- Ernest Lin
- Wesley Nguyen
- Ethan Lee
- Stephen Kim

<a id='research_question'></a>
# Research Question

Do the macroscopic socioeconomic features of a state, specifically median income, percentage of population without health insurance, and labor breakdown, have a correlation to COVID mortality rate in 2020-2021?

# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION